In [1]:
from config import QDRANT_DB_API_KEY, QDRANT_URL, OPEN_AI_API_KEY
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from openai import OpenAI

/Users/nihitagarwal/Desktop/Hacklytics_app/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Connection for semantic queries
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_DB_API_KEY,
)


In [3]:
encoder = SentenceTransformer('all-MiniLm-L6-v2')

In [4]:
# Connection to OpenAI API
client = OpenAI(
  organization='org-Cl2hWJbfjhnqQWMc1MTRzyqy',
  api_key=OPEN_AI_API_KEY
)

In [55]:
def make_query(prompt):
    if prompt != None:
        hits = qdrant_client.search(
        collection_name="my_hotels",
        query_vector=encoder.encode(prompt).tolist(),
        limit=5
        )
        return hits
    else:
        return None

In [47]:
def generate_response(prompt, db_response):
    conversation = []
    new_prompt = f'Give a JSON with fields -  "hotel_name" and "reason" (which states in 2 sentences why the hotel matches prompt) for each unique hotel  that satisfies the user prompt- "{prompt}" using information below .\n"""'
    for hotel in db_response:
        new_prompt = new_prompt + '\n#' + str(hotel.payload) + " hotel id: " + str(hotel.id)
    new_prompt += '"""'
    conversation.append({"role":"system", "content":"You are a hotel advisor."})
    conversation.append({"role": "user", "content":new_prompt})
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=conversation
    )
    return response

In [56]:
prompt = 'Big swimming pool in hotel in America'


In [57]:
db_response = make_query(prompt)
db_response

[ScoredPoint(id=3637, version=170, score=0.4759348, payload={'country': 'United States', 'hotel_description': "A tropical oasis in New York City's world-famous Times Square, Margaritaville Resort Times Square invites you to set your watch to island time. Whether you're kicking back in your breezy room or suite, lounging poolside on the rooftop deck with a frozen concoction in hand, or taking a culinary tour through the Caribbean at our restaurants, this relaxing retreat is your passport to paradise. And when you're ready for adventure, you'll find many of the city's most famous attractions within walking distance, including the theaters of Broadway, Central Park and the iconic Empire State Building.", 'hotel_image': 'https://media-cdn.tripadvisor.com/media/photo-s/28/38/de/a8/ground-floor-lobby.jpg', 'hotel_name': 'Margaritaville Resort Times Square', 'hotel_url': 'https://www.tripadvisor.com/Hotel_Review-g60763-d23462501-Reviews-or30-Margaritaville_Resort_Times_Square-New_York_City_Ne

In [58]:
response = generate_response(prompt, db_response)
response

ChatCompletion(id='chatcmpl-8qsiMusKajizuQAPunfJrRmWglBRF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n   "hotels":[\n      {\n         "hotel_name":"Margaritaville Resort Times Square",\n         "reason":"The hotel offers a big swimming pool on the rooftop deck where guests can relax and enjoy a frozen concoction while lounging poolside. Additionally, it is located in New York City, United States."\n      }\n   ]\n}', role='assistant', function_call=None, tool_calls=None))], created=1707614070, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=72, prompt_tokens=2860, total_tokens=2932))

In [59]:
print(response.choices[0].message.content)

{
   "hotels":[
      {
         "hotel_name":"Margaritaville Resort Times Square",
         "reason":"The hotel offers a big swimming pool on the rooftop deck where guests can relax and enjoy a frozen concoction while lounging poolside. Additionally, it is located in New York City, United States."
      }
   ]
}


In [64]:
import ast
hotels = ast.literal_eval(response.choices[0].message.content)
hotels['hotels']

[{'hotel_name': 'Margaritaville Resort Times Square',
  'reason': 'The hotel offers a big swimming pool on the rooftop deck where guests can relax and enjoy a frozen concoction while lounging poolside. Additionally, it is located in New York City, United States.'}]